In [ ]:
import findspark
import pyspark
from pyspark.sql import SparkSession, Row, Window
import pandas as pd
from collections import OrderedDict
import json, xmltodict
import pyspark.sql.functions as F
import os

In [ ]:
# Create new Sparkcontext instance for analysis
findspark.init()
sc = pyspark.SparkContext(appName="FB_Messenger", master='local')
spark = SparkSession.builder.getOrCreate()

In [ ]:
### v5 ###

# Total Messages: 430,212
# Expected Runtime: 4.55 minutes

# Path pointing at FB messages main directory (1076 relevant files as of now)
path_of_the_fb_message_inbox = r'C:\Users\RajNa\Downloads\FB_Raj_12212021\messages\inbox'

count = 0

# Final DF to contain all Messages
fb_message_cols = ["content", "sender_name", "timestamp_ms", "thread_type", "title"]
fb_message_default_vals = [('Message', 'Sender', 1234, 'Type', 'title')]
fb_messages_all_df = spark.createDataFrame(fb_message_default_vals,fb_message_cols)


# Parse through each folder in the Facebook Inbox folder
for fb_chat in os.listdir(path_of_the_fb_message_inbox):
    
        # Parse through each subfolder (representing each indvidual or group chat)
        for fb_chat_details in os.listdir(os.path.join(path_of_the_fb_message_inbox,fb_chat)):
            
            # Only process message_ files (not photos, videos, etc)
            if 'message_' in fb_chat_details:

                # Read in entire JSON file (multiline)
                fb_chat_df = spark.read.option("multiline","true") \
                .json(os.path.join(path_of_the_fb_message_inbox,fb_chat, fb_chat_details))
                
                # Drop Unneeded Columns
                fb_chat_df = fb_chat_df.drop('is_still_participant','magic_words','thread_path')
                
                # Use explode to get a row for every single message, and filter out uneeded columns 
                # A fail implies a missing column, and so can be ignored
                try:
                    fb_chat_df = fb_chat_df \
                    .select('thread_type', 'title', F.explode('messages')) \
                    .select("col.*", "*") \
                    .select('content', 'sender_name', 'timestamp_ms', 'thread_type', 'title')                   
                except:
                    count = count
                
                # If it already exists, union the newest chat to a dataframe with all messages in it
                try:
                    fb_messages_all_df = fb_messages_all_df.union(fb_chat_df)  
                # An error indicates a bad join due to missing column, probably no content, and thus can be ignored
                except:
                    count = count

                count = count + 1
                print(count)